For more details of this implementation. Check out ObjectDetection_with_images Notebook

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
 
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# Importing OpenCV to capture video from webcam
import cv2
cap = cv2.VideoCapture(0)

In [2]:
sys.path.append("models/research/")
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [44]:
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
#MODEL_NAME = 'faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28'

MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = "http://download.tensorflow.org/models/object_detection/"

PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'
 
PATH_TO_LABELS = os.path.join('models', 'research', 'object_detection', 'data', 'mscoco_label_map.pbtxt')
 
NUM_CLASSES = 90

In [45]:
"""opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
    file_name = os.path.basename(file.name)
    if 'frozen_inference_graph.pb' in file_name:
        tar_file.extract(file, os.getcwd())"""

"opener = urllib.request.URLopener()\nopener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)\n\ntar_file = tarfile.open(MODEL_FILE)\nfor file in tar_file.getmembers():\n    file_name = os.path.basename(file.name)\n    if 'frozen_inference_graph.pb' in file_name:\n        tar_file.extract(file, os.getcwd())"

In [46]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [47]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [26]:
""" This block is not needed for video
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)"""

"""PATH_TO_TEST_IMAGES_DIR = 'test_images'
num_images = 2
IMAGE_SIZE = (12, 8)
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, num_images) ]"""

# Capturing video and detecting object
- Tensorflow version of the code
- This method is very fast compared to the previous inference version

In [7]:
obj_to_width = {
    'bottle' : 7.5,
    'cup' : 10,
    'chair' : 58,
    'apple' : 8,
    'orange' : 8,
    'tv' : 102,
    'mouse' : 6.5,
    'laptop' : 37.5,
    'remote' : 6,
    'cell phone' : 7,
    'book' : 13
}

id_to_obj = {}
for i in range(len(categories)):
    id_to_obj[categories[i]['id']] = categories[i]['name'];

In [37]:
def find_distance(known_width, focal_length, pixel_width):
    return (known_width*focal_length)/pixel_width

In [38]:
def write_distances_on_image(image, boxes, scores, classes, id_to_obj, obj_to_width, focal_length = 1035):
    num_obj = sum(sum(scores > 0.5))
    
    for i in range(num_obj):
        x1 = int(boxes[0][i][1]*800)
        y1 = int(boxes[0][i][0]*600)
        x2 = x1 + 250
        y2 = y1 + 30
        
        ## Block to find the distance
        px_width = (boxes[0][i][3] - boxes[0][i][1])*800
        px_height = (boxes[0][i][2] - boxes[0][i][0])*600
        
        pxl = min(px_width, px_height)
        
        obj = id_to_obj[classes[0][i]]
        try:
            rl_width = obj_to_width[obj]
            dis = int(find_distance(rl_width, focal_length, pxl))
                
        except KeyError:
            dis = 0
        ## Block Ends here    
        
        
        image = cv2.rectangle(image, (x1, y1), (x2, y2), color = (255, 255, 255), thickness = -1)
        image = cv2.putText(image, "dis : " + str(dis) + " cms", (x1+15, y1+20), cv2.FONT_HERSHEY_SIMPLEX, fontScale = 0.8, color = (0,0,0), thickness = 2)
        
    return image

In [48]:
cap = cv2.VideoCapture(0)

with detection_graph.as_default():
    with tf.compat.v1.Session(graph=detection_graph) as sess:
        while True:
            ret, image_np = cap.read()
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            # Each box represents a part of the image where a particular object was detected.
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            # Each score represent how level of confidence for each of the objects.
            # Score is shown on the result image, together with the class label.
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            # Actual detection.
            (boxes, scores, classes, num_detections) = sess.run(
              [boxes, scores, classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=8)
            
            image_np = write_distances_on_image(image_np, boxes, scores, classes, id_to_obj, obj_to_width, focal_length = 1035)
                        
            cv2.imshow('object detection', cv2.resize(image_np, (800,600)))
            if cv2.waitKey(5) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                cap.release()
                break

# Below is the inference version of the implementation
- We had used this method in our implementation with images
- But this method is **very slow**

In [7]:
def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.compat.v1.Session() as sess:
        # Get handles to input and output tensors
            ops = tf.compat.v1.get_default_graph().get_operations()
            all_tensor_names = {output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
              'num_detections', 'detection_boxes', 'detection_scores',
              'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.compat.v1.get_default_graph().get_tensor_by_name(tensor_name)
                    
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.compat.v1.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
              'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

In [11]:
while True:
    ret, image_np = cap.read()
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
    
    cv2.imshow('object detection', cv2.resize(image_np, (800,600)))
    if cv2.waitKey(5) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break